# Interactive Geomap of Point Reyes Snowy Plovers Study Area


*   Each bubble represents one location (ex. Drake's Beach, Limantour, North Beach, etc)
*   Size of bubble reflects an overall plover activity index (combining plovers observed, nests, nest success), so larger circles mean higher plover activity
* When you click on a bubble, location name, plovers observed, mean nest success, and number of nests pops up
* Overall this map highlights spatial hotspots where plovers are most active and where the nest success is high or low
* Helps future ML models because it shows which beach contributes most to plover productivity


In [71]:
!pip install folium
!pip install geopy


In [72]:
import pandas as pd
import folium
import numpy as np

#Load the cleaned dataset
csv_path = "/content/drive/MyDrive/Snowy Plover Datasets/cleaned_plover_dataset.csv"
df = pd.read_csv(csv_path)
df.head(1)

,ID,SNPL_Data_ID,Event_ID,Type,Basis_Of_Record,Unit_Code,Location_ID,Loc_Code,Loc_Name,Start_Year,...,Eggs,Hatchlings,Fledglings,Failure_Reason,Predator_Type,Restored_Area,NestSuccess,NumChicks_Banded,MeanChickWeight_g,Pct_Chicks_EggTooth
0,11975,002b83c1-799d-42da-9541-9d9c992478c9,90d84c98-4f70-4b19-b352-cd2bd3bfc5f4,event,HumanObservation,PORE,{764DC5D8-DDE7-49E6-B146-A9328A9BB8B6},NP,North Beach - Abbott's,2024,...,0,0,0,Unknown,Unknown,False,False,0,5.95,0.666667


In [73]:
# Keep rows with coordinates
df_geo = df.dropna(subset=["Decimal_Latitude", "Decimal_Longitude"])

# Drop rows without location name
df_geo = df_geo.dropna(subset=["Loc_Name"])

In [74]:
# Aggregate by beach, so there will be one row per Loc name with coordinates, plovers observed, mean nest success, and # of unique nests


location_stats = (
    df_geo
    .groupby("Loc_Name")
    .agg(
        mean_lat=("Decimal_Latitude", "mean"),
        mean_lon=("Decimal_Longitude", "mean"),
        total_plovers=("Total_SNPL", "sum"),
        mean_nest_success=("NestSuccess", "mean"),
        num_nests=("Nest_ID", "nunique")
    )
    .reset_index()
)

location_stats.head()


,Loc_Name,mean_lat,mean_lon,total_plovers,mean_nest_success,num_nests
0,2011 Dune Restoration Area,38.106618,-122.957024,1237,0.225392,78
1,Drakes Beach,38.031467,-122.934221,35,0.615385,1
2,Kehoe Beach,38.132174,-122.953979,6742,0.167665,86
3,Limantour Beach,38.027771,-122.899692,6144,0.162562,43
4,North Beach - Abbott's,38.092897,-122.967586,14555,0.233197,268


In [75]:
# Replace NaN if there are any

location_stats["total_plovers"] = location_stats["total_plovers"].fillna(0)

location_stats["mean_nest_success"] = location_stats["mean_nest_success"].fillna(0)

location_stats["num_nests"] = location_stats["num_nests"].fillna(0)


In [76]:
#Center for map
center_lat = location_stats["mean_lat"].mean()
center_lon = location_stats["mean_lon"].mean()

print("Center for Map:", center_lat, center_lon)

# Create basic Folium map
plover_map = folium.Map(location=[center_lat, center_lon], zoom_start=10)
plover_map


# Create an "activity index" based on total plovers per location
max_plovers = location_stats["total_plovers"].max()
if max_plovers == 0:

    location_stats["activity_index"] = 0
else:

    location_stats["activity_index"] = location_stats["total_plovers"] / max_plovers



Center for Map: 38.055887931594796 -122.95449740738675


In [77]:
# Add bubbles for bubble map

for i in range(len(location_stats)):
    row = location_stats.iloc[i]

    lat = row["mean_lat"]
    lon = row["mean_lon"]
    loc_name = row["Loc_Name"]
    total_plovers = int(row["total_plovers"])
    mean_nest_success = row["mean_nest_success"]
    num_nests = int(row["num_nests"]) if not np.isnan(row["num_nests"]) else 0
    activity_index = row["activity_index"]


    radius = activity_index * 3000

    #label
    #popup_text = (
        #f"{loc_name}<br>"
        #f"Total plovers observed: {total_plovers}<br>"
        #f"Mean nest success: {mean_nest_success:.2f}<br>"
        #f"Number of nests: {num_nests}"
    #)

    #Changed to make the text box bigger

    popup = folium.Popup(
      f"<b>{loc_name}</b><br>"
      f"Total plovers observed: {total_plovers}<br>"
      f"Mean nest success: {mean_nest_success:.2f}<br>"
      f"Number of nests: {num_nests}",
      max_width=250
)

    folium.Circle(
        location=[lat, lon],
        popup = popup,
        radius = radius,
        color = "pink",
        fill = True,
        fill_color = "pink",
        fill_opacity = 0.5,
    ).add_to(plover_map)

plover_map

In [79]:
# save as HTML
plover_map.save("plover_activity_bubble_map.html")

output_path = "/content/drive/MyDrive/Snowy Plover Datasets/plover_map.html"
plover_map.save(output_path)
output_path



'/content/drive/MyDrive/Snowy Plover Datasets/plover_map.html'